# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_df_to_load = "../WeatherPy/weather_df.csv"
weather_data = pd.read_csv(weather_df_to_load)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# config gmaps
gmaps.configure(api_key=g_key)

In [10]:
locations = weather_data[['Lat', 'Lng']]
humidity = weather_data["Humidity"].astype(float)

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_humidity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
mild_climate = weather_data.loc[(weather_data["Wind Speed"] < 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

mild_climate

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,20,Cayenne,0,GF,1608171129,94,4.93,-52.33,75.20,3.36
221,221,La Reforma,0,MX,1608171157,95,18.10,-91.02,71.42,5.41
229,229,Prieska,0,ZA,1608171158,20,-29.66,22.75,75.43,1.81
234,234,Presidencia Roque Sáenz Peña,0,AR,1608171159,71,-26.79,-60.44,77.00,1.01
242,242,Pilar,0,BR,1608171160,94,-9.60,-35.96,73.40,3.36
300,300,Veinticinco de Mayo,0,AR,1608171168,46,-35.43,-60.17,70.00,8.41
326,326,Comodoro Rivadavia,0,AR,1608171156,33,-45.87,-67.50,73.40,9.17
420,420,Sittwe,0,MM,1608171007,55,20.15,92.90,72.37,6.24
437,437,Nikki,0,BJ,1608171190,23,9.94,3.21,70.66,4.70
455,455,Boussé,0,BF,1608171192,27,12.66,-1.89,78.80,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = mild_climate.loc[:,["City","Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
20,Cayenne,GF,4.93,-52.33,
221,La Reforma,MX,18.10,-91.02,
229,Prieska,ZA,-29.66,22.75,
234,Presidencia Roque Sáenz Peña,AR,-26.79,-60.44,
242,Pilar,BR,-9.60,-35.96,
300,Veinticinco de Mayo,AR,-35.43,-60.17,
326,Comodoro Rivadavia,AR,-45.87,-67.50,
420,Sittwe,MM,20.15,92.90,
437,Nikki,BJ,9.94,3.21,
455,Boussé,BF,12.66,-1.89,


In [25]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [26]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"


    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()

    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

print("------------End of Search--------------")

Retrieving Results for Index 20: Cayenne.
Closest hotel in Cayenne is Royal Amazonia.
------------
Retrieving Results for Index 221: La Reforma.
Missing field/result... skipping.
------------
Retrieving Results for Index 229: Prieska.
Closest hotel in Prieska is Allegro Guest House.
------------
Retrieving Results for Index 234: Presidencia Roque Sáenz Peña.
Closest hotel in Presidencia Roque Sáenz Peña is Hotel Gualok.
------------
Retrieving Results for Index 242: Pilar.
Missing field/result... skipping.
------------
Retrieving Results for Index 300: Veinticinco de Mayo.
Closest hotel in Veinticinco de Mayo is Gran Hotel 25 de Mayo.
------------
Retrieving Results for Index 326: Comodoro Rivadavia.
Closest hotel in Comodoro Rivadavia is Lucania Palazzo Hotel.
------------
Retrieving Results for Index 420: Sittwe.
Closest hotel in Sittwe is Yuzana Aung Hotel.
------------
Retrieving Results for Index 437: Nikki.
Closest hotel in Nikki is Hotel John.
------------
Retrieving Results for

In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [32]:
# add marker
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

fig


Figure(layout=FigureLayout(height='420px'))